# **Pseudopotential**

<i class="fa fa-home fa-2x"></i><a href="./index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/pseudopotential.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
    The pseudopotential is an effective potential constructed to replace the 
    complicated effects of the core electrons. This notebook presents how 
    pseudopotentials are constructed. We used the wavefunctions from the solution
    of the Shrödinger equation of one hydrogen atom.
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand why we need to use the pseudopotentials.</li>
        <li> Know how to construct the pseudopotentials by Kerker's method.</li>
        <li> Examine the cutoff distance.</li>
    </ol>

</details>

## **Background theory**

<p style="text-align: justify;font-size:15px">
    There are many methods to construct the pseudopotentials. 
    In 1979, Hamman,Schlüter, and Chiang introduced the norm-conserving
    pseudopotentials. Here, we presented
    one simple norm-conserving method, which was inivited by G. P. Kerker, 
    <a href="https://doi.org/10.1088/0022-3719/13/9/004">
    J. Phys. C 13, L189 (1980).</a>
    The theory and method is also well written in the chapter 5 of the book
    <a href="https://www.sciencedirect.com/book/9780123044600/solid-state-physics">
    "Soild State Physics (Second Edition)"</a>
    by Giuseppe Grosso and Giuseppe Pastori Parravicini.
</p>

<details close>
<summary style="font-size: 20px">Polynomial representation</summary>
<p style="text-align: justify;font-size:15px">
    The raidal Schrödinger equation is given as:
</p>

$\large \left[
  -\dfrac{1}{2m} \, \dfrac{\partial^2}{\partial r^{2}} + 
  \dfrac{l(l+1)}{2r^2} + V(r)\right] r\psi(r) = Er\psi(r)$ (1)
  
<p style="text-align: justify;font-size:15px">
    The top figure in the below shows the $\Psi(r)r$ and pseudowavefunction $\Psi^{PS}(r)r$.
    We need choose a resonable cutoff distance to sperate the core region and the rest.
    You can use the slider to change the cutoff distance $R_c$. We need to replace the 
    core region wavefunction with a smoonth and nodeless function. Here, we construct the
    pseudo wavefunction as:
</p>


$ \large  \Psi^{pseudo}(r) =
    \begin{cases}
      r^le^{p(r)}  & \text{for r < r$_c$} \\
      \Psi(r) & \text{for r > r$_c$}
    \end{cases}       
$

<p style="text-align: justify;font-size:15px">
    where $p(r)$ is a polynomial defined as:
</p>

$ \large p(r) = \lambda_0 + \sum_{2}^{n} \lambda_n r
$

<p style="text-align: justify;font-size:15px">
    In this polynomial, the 1st order coefficient $\lambda_1$ is set to be zero, 
    which presents the pseudopotential being singular at $r=0$. In this notebook,
    we choice to use the 4th order polynomial.
</p>

$ \large p(r) = \lambda_0 + \lambda_2 r^2 + \lambda_3 r^3 + \lambda_4 r^4 $
</details>

<details close>
<summary style="font-size: 20px">Conditions for the pseudo wavefunction</summary>

<p style="text-align: justify;font-size:15px">
    We need four equations to obtain the coefficients ($\lambda$). The equations
    are formulated with two conditions.
</p>

<p style="text-align: justify;font-size:17px; font-weight: bold;">
    1. continuity of function at r$_c$
</p>

<p style="text-align: justify;font-size:15px">
    The pseudo wavefunction needs keep continuity at r$_c$. The zero, 1st and
    2nd left derivatives at r$_c$ should equal the respective derivatives at
    right. Here, we can formulate three equations.
</p>
    
<p style="text-align: justify;font-size:17px; font-weight: bold;">
    2. Norm-Conserving
</p>

<p style="text-align: justify;font-size:15px">
    The norms of the wavefunction and pseudo wavefunction should be the same.
    Since when $r > r_c$ the functions are the same, we only need to integrate
    from zero to r$_c$.
</p>

$\Large \int_0^{r_c} \Psi(r)^2r^2 dr = \int_0^{r_c} \Psi^{PS}(r)^2r^2 dr
$

<p style="text-align: justify;font-size:15px">
    which also means the yellow area as shown in the figure below should be
    equal to the green area.
</p>
</details>
    
<details close>
<summary style="font-size: 20px">Obtain the pseudopotential</summary>
<p style="text-align: justify;font-size:15px">
    In the final step, the pseudopotential is computed from the inversed Shrödinger equation.
</p>

$\large V^{PS}(r) = E - \dfrac{l(l+1)}{2r^2} + \dfrac{1}{2\Psi^{PS}(r)}
\dfrac{\partial^2 \Psi^{PS}(r)}{\partial r^{2}}$
</details>

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
    <li> Why we need pseudopotentials?
    <details style="color: blue">
    <summary>Hints</summary>
        The wavefunction oscillate rapidly in the core region, 
        which needs more Fourier modes to describe.
    </details>
    </li>
    <li> What are the advantages of the norm conserving pseudopotentials?
    <details style="color: blue">
    <summary>Hints</summary>
        We need to reproduce the pseudo-charge density as accurately as possible.
    </details>
    </li>
    <li> Are the pseudopotentials local or not?
    <details style="color: blue">
    <summary>Hints</summary>
        The pseudopotentials also dependent on the quantum number n and l.
    </details>
    </li>
    <li> If we want to add one more term $\lambda_5r^5$ to the polynomial, how should
        we solve it?
    <details style="color: blue">
    <summary>Hints</summary>
        We need add one more equation to get the coefficients. We can consider higher
        continuity of the wavefunction (3rd derivative).
    </details>
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
from sympy.physics.hydrogen import R_nl, E_nl
from sympy.abc import r
from sympy.functions import exp
from sympy import lambdify, diff, log
import numpy as np
import matplotlib.pyplot as plt
from sympy.solvers.solveset import linsolve
from ipywidgets import FloatSlider, Button, IntSlider, Layout, HBox, VBox, Label, Tab, Layout, Text
from scipy.optimize import newton
import matplotlib.gridspec as gridspec

%matplotlib widget

In [ ]:
# sn: the slider to control the quantum number n
# sl: the slider to control the quantum number l
sn = IntSlider(value=3, min=1, max=5, description="n")
sl = IntSlider(value=0, min=0, max=sn.value-1, description="l")

labeln = Label(value="(The principal quantum number n)")
labell = Label(value="(The angular quantum number l)")

#init the quantum number n and l
n = sn.value
l = sl.value
Z = 1

#ho: is the Hydrogen radial wavefunction
#Ea: is the eigenvalue
#rf: make the analytical formula numerically
ho = R_nl(n, l, r, Z=Z)
Ea = E_nl(n, Z=Z)
rf = lambdify(r, ho, "numpy")

def nvalue_change(c):
    """Observe the change of the sn and update plot.
    """
    global n, ho, Ea, rf
    n = c["new"]
    sl.value = 0
    sl.max = n - 1
    ho = R_nl(n, l, r, Z=Z)
    Ea = E_nl(n, Z=Z)
    rf = lambdify(r, ho, "numpy")
    update_plot()

sn.observe(nvalue_change, names="value")

def lvalue_change(c):
    """Observe the change of the sl and upate plot.
    """
    global l, ho, Ea, rf
    l = c["new"]
    ho = R_nl(n, l, r, Z=Z)
    Ea = E_nl(n, Z=Z)
    rf = lambdify(r, ho, "numpy")
    update_plot()

sl.observe(lvalue_change, names="value")

In [ ]:
s_rc = FloatSlider(value = 13.0, min = 0.0, max = 50, description = "Cutoff $R_c$ : ", 
                   layout={'width':'550px'})

compute = Button(description="Compute pseudopotential", style={'description_width': 'initial'})
flip = Button(description="Flip")

label_flip = Label(value=r"(Due to the formula of the polynomial, $\Psi(r_c)r_c$ must be positve."+ 
                   " Flip the wavefunction to fit.)")


text_l0 = Text(description = "Left $\Psi^{(0)}(R_c)$:", style = {'description_width': 'initial'})
text_l1 = Text(description = "Left $\Psi^{(1)}(R_c)$:", style = {'description_width': 'initial'})
text_l2 = Text(description = "Left $\Psi^{(2)}(R_c)$:", style = {'description_width': 'initial'})

text_r0 = Text(description = "Right $\Psi^{(0)}(R_c)$:", style = {'description_width': 'initial'})
text_r1 = Text(description = "Right $\Psi^{(1)}(R_c)$:", style = {'description_width': 'initial'})
text_r2 = Text(description = "Right $\Psi^{(2)}(R_c)$:", style = {'description_width': 'initial'})

cof_0 = Text(description = "$\lambda_0$:", style = {'description_width': 'initial'})
cof_2 = Text(description = "$\lambda_2$:", style = {'description_width': 'initial'})
cof_3 = Text(description = "$\lambda_3$:", style = {'description_width': 'initial'})
cof_4 = Text(description = "$\lambda_4$:", style = {'description_width': 'initial'})

def clear_texts():
    text_l0.value = ""
    text_l1.value = ""
    text_l2.value = ""
    text_r0.value = ""
    text_r1.value = ""
    text_r2.value = ""
    cof_0.value = ""
    cof_2.value = ""
    cof_3.value = ""
    cof_4.value = ""

output1 = VBox([HBox([text_l0, text_r0]), HBox([text_l1, text_r1]), HBox([text_l2, text_r2])]);
output2 = VBox([cof_0, cof_2, cof_3, cof_4])

tab = Tab(layout=Layout(width='650px'))
tab.children = [output1, output2]
tab.set_title(0, r"Continuity at Rc")
tab.set_title(1, r"poly. coefficients")

display(HBox([sn, labeln]), HBox([sl, labell]), HBox([flip, label_flip]))
display(HBox([s_rc, compute]))
display(tab)

def on_flip(c):
    """On click event for the Flip button.
    """
    global ho, rf
    ho = -ho
    rf = lambdify(r, ho, "numpy")
    update_plot()
    
flip.on_click(on_flip)

In [ ]:
img = plt.figure(tight_layout=True, figsize=(7,7))
img.canvas.header_visible = False

gs = gridspec.GridSpec(2, 1)

ax1 = img.add_subplot(gs[0, 0])
ax2 = img.add_subplot(gs[1, 0])

x1 = np.arange(0, 50.0, 0.01)
y1 = rf(x1)*x1

line_rho, = ax1.plot(x1, y1, 'r-', label="$\Psi(r)r$")
ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
ax1.set_xlim([0, 50.0])

ax1.hlines(0, 0, 50, 'k','--')

line_rc1 = ax1.axvline(s_rc.value)
line_pswf, = ax1.plot([],[],'b-', linewidth=1.5, label="$\Psi^{PS}(r)r$")
ann_rc = ax1.annotate("$R_c$", xy=(s_rc.value + 1.0, rf(s_rc.value)*s_rc.value), fontsize=20)
point, = ax1.plot(s_rc.value, rf(s_rc.value)*s_rc.value, 'ko')

ann_norm1 = ax1.annotate("Yellow area:", xy=(200, 40), xycoords='axes points', fontsize=12, 
                         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
ann_norm2 = ax1.annotate("Green area:", xy=(200, 15), xycoords='axes points', fontsize=12,
                         bbox=dict(boxstyle='round', facecolor='green', alpha=0.5))

ax1.set_xlabel("r", fontsize = 15)
ax1.set_ylabel("$\Psi(r)r$", fontsize = 15)

ax1.legend(fontsize=15)
#ax1.set_ylim([-0.3, 0.5])


x2 = np.linspace(0.001, 50, 500);
y2 = -Z/x2;

ax2.plot(x2, y2, 'r--', linewidth=1.0, label="$-Z/r$")

ax2.set_xlim([0, 50.0])
#ax2.set_ylim([-1.0, 0.10])
line_rc2 = ax2.axvline(s_rc.value)
line_psv, = ax2.plot([],[], 'b-', linewidth=1.5, label="$V^{SP}(r)$")
ax2.hlines(0, 0, 50, 'k','--')

ax2.set_xlabel("r", fontsize = 15)
ax2.set_ylabel("$V$", fontsize = 15)

ax2.legend(fontsize=15)

def update_plot():
    """ Update the plot when quantum number n and l changing.
    """
    x1 = np.arange(0, 50.0, 0.01)
    y1 = rf(x1)*x1
    line_rho.set_data([x1, y1])
    line_rc1.set_data(s_rc.value, [-1, 1])
    line_rc2.set_data(s_rc.value, [-1, 1])
    ann_rc.set_position((s_rc.value + 1.0, rf(s_rc.value)*s_rc.value))
    ax1.collections.clear()
    ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
    ax1.hlines(0, 0, 50, 'k','--')
    line_pswf.set_data([],[])
    line_psv.set_data([],[])
    point.set_data(s_rc.value, rf(s_rc.value)*s_rc.value)
    ax1.set_ylim([y1.min()-0.04, y1.max()+0.04])
    clear_texts()

def on_rc_change(b):
    """ Update the plot when the slider of the Rc changing.
    """
    if rf(s_rc.value) <= 0:
        compute.disabled = True
    else:
        compute.disabled = False
    x1 = np.arange(0, 50.0, 0.01)
    y1 = rf(x1)*x1
    line_rc1.set_data(s_rc.value, [-1, 1])
    line_rc2.set_data(s_rc.value, [-1, 1])
    ann_rc.set_position((s_rc.value + 1.0, rf(s_rc.value)*s_rc.value))
    ax1.collections.clear()
    ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
    ax1.hlines(0, 0, 50, 'k','--')
    line_pswf.set_data([],[])
    line_psv.set_data([],[])
    point.set_data(s_rc.value, rf(s_rc.value)*s_rc.value)
    clear_texts()
     
s_rc.observe(on_rc_change, names='value')

In [ ]:
def compute_right_derivative(rc):
    k0 = rf(rc)
    k1 = diff(ho, r).subs(r, rc).evalf()
    k2 = diff(ho, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])

def compute_left_derivative(ps, rc):
    psf = lambdify(r, ps, "numpy")
    k0 = psf(rc)
    k1 = diff(ps, r).subs(r, rc).evalf()
    k2 = diff(ps, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])
    
def solver_kernel(devs, rc, l, b):
    A = np.zeros([3,4])
    A[0, :] = np.array([1, rc**2, rc**3, rc**4]);
    A[1, :] = np.array([0, 2*rc, 3*rc**2, 4*rc**3]);
    A[2, :] = np.array([0, 2, 6*rc, 12*rc**2])
    
    B = np.zeros(3)
    B[0] = log(devs[0]/rc**l);
    B[1] = devs[1]/devs[0] - l/rc;
    B[2] = devs[2]/devs[0] - devs[1]**2/devs[0]**2 + l/rc**2;
    
    B-=b*A[:, 1];
    A = np.delete(A, (1), axis=1);
    
    coff = np.linalg.solve(A, B);
    coff = np.insert(coff, 1, b)
    
    return coff

def diff_norms(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps, "numpy")
    psr = lambdify(r, ps*ps*r*r, "numpy")
    hor = lambdify(r, ho*ho*r*r, "numpy")
        
    x1 = np.linspace(0, rc, 800);
        
    norm1 = np.sum(hor(x1))*(x1[1]-x1[0])
    norm2 = np.sum(psr(x1))*(x1[1]-x1[0])
    
    return float(norm1 - norm2)

def compute_norms(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps, "numpy")
    psr = lambdify(r, ps*ps*r*r, "numpy")
    hor = lambdify(r, ho*ho*r*r, "numpy")
        
    x1 = np.linspace(0, rc, 800);
        
    norm1 = np.sum(hor(x1))*(x1[1]-x1[0])
    norm2 = np.sum(psr(x1))*(x1[1]-x1[0])
    
    return norm1, norm2

def compute_potential(l):
    psf = Ea - l*(l+1)/(2*r*r) + 1/(2*ho*r)*diff(ho*r, r, r)
    return lambdify(r, psf, "numpy")


def plot_ps_wavefunction(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps*r, "numpy")
    
    devl = compute_left_derivative(ps, rc)
    
    x1 = np.linspace(0, rc, 800);
    line_pswf.set_data(x1, psf(x1));
    ax1.fill_between(x1, psf(x1), 0, where=x1<s_rc.value, facecolor='green', alpha=0.5)
    
    text_l0.value = str(ps.subs(r, rc).evalf())
    text_l1.value = str(diff(ps, r).subs(r, rc).evalf())
    text_l2.value = str(diff(ps, r, r).subs(r, rc).evalf())
    
    text_r0.value = str(ho.subs(r, rc).evalf())
    text_r1.value = str(diff(ho, r).subs(r, rc).evalf())
    text_r2.value = str(diff(ho, r, r).subs(r, rc).evalf())
    
    cof_0.value = str(coff[0])
    cof_2.value = str(coff[1])
    cof_3.value = str(coff[2])
    cof_4.value = str(coff[3])

def plot_ps_potential(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    pf  = r**(l+1)*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = Ea - l*(l+1)/(2*r*r) + 1/(2*pf)*diff(pf, r, r)
    psfnl = lambdify(r, psf, "numpy")
    psfnr = compute_potential(l)
        
    x1 = np.linspace(0.001, rc, 800);
    x2 = np.linspace(rc, 50, 800);
    line_psv.set_data(np.concatenate((x1,x2)), np.concatenate((psfnl(x1),psfnr(x2))));
    ax2.set_ylim([psfnl(x1).min(axis=0)-0.05, max(psfnl(x1).max(axis=0)+0.1, 0.05)])

def compute_pseudopotential(c):
    try:
        b = newton(lambda x: diff_norms(x, s_rc.value, l), x0 = 0.00001, tol = 1e-10, maxiter=100)
        
        if abs(diff_norms(b, s_rc.value, l)) > 0.001:
            ann_norm1.set_text("No numerical solution found!");
            ann_norm2.set_text("Please change the $r_c$!");
            return None            
    except:
        pass
        ann_norm1.set_text("No numerical solution found!");
        ann_norm2.set_text("Please change the $r_c$!");
        return None
    plot_ps_wavefunction(b, s_rc.value, l)
    plot_ps_potential(b, s_rc.value, l)
    norm1, norm2 = compute_norms(b, s_rc.value, l)
    ann_norm1.set_text("Yellow area square:" + str("{:.10f}".format(norm1)))
    ann_norm2.set_text("Green area square:" + str("{:.10f}".format(norm2)))
    
compute.on_click(compute_pseudopotential)
compute_pseudopotential("init");

<details>
    <summary style="font-size: 22px;"><b>Legend</b></summary>

<p style="text-align: justify;font-size:15px">
    Here, our employed the wavefunctions from the solution of Shrödinger
    equation of one hydrogen atom. One can choose the state with quantum
    number n and l by the sliders.
</p>

<p style="text-align: justify;font-size:15px">
    Since the formula of the pseudo wavefunction is $r^le^{p(r)}$, $\Psi(R_c)R_c$
    must be a positive number. Click the "Flip" button to flip the wavefuntion
    and make the selected $\Psi(R_c)R_c$ value positive.
</p>
    
<p style="text-align: justify;font-size:15px">
    The position of the cutoff distance $R_c$ can be tunned with the slider. 
    Click the "Compute pseudopotentials" button to obtain the pseudo wavefunction
    and pseudopotentials. The up figure shows the calculated pseudo radial 
    wavefunction time r $\Psi^{PS}(r)r$ (in blue line) and radial wavefunction 
    time r $\Psi(r)r$ (in red line). In order to fullfil the norm-conserving
    condition, the green and yellow areas should be equal. The square of the area
    should be 1 when r goes to $+\infty$. The bottom figure shows the calculated
    pseudopotentials.
</p>
    
<p style="text-align: justify;font-size:15px">
    The pseudo wavefunctions are calculated by formulating the continuity of the 
    functions. The zero, first and second derivatives of the wavefunction at $R_c$
    should be the same. The derivations at $R_c$ and coefficients are presented
    in the textbox.
</p>
</details>